In [1]:
from tensorflow import keras
from keras import models
from keras import layers
from keras import optimizers, losses, metrics

import numpy as np

Using TensorFlow backend.


# 데이터 생성

In [2]:
# 캐릭터 글자 목록
char_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
             'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

# 캐릭터 사전 생성
char_to_idx = {c: i for i, c in enumerate(char_list)}
dic_len = len(char_to_idx)

print(char_to_idx)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [3]:
# 단어 목록
# 앞의 세 글자가 주어지면 마지막 글자를 예측
# lov -> e
word_list = ['love', 'look', 'face', 'fast', 'home', 'hope',
             'good', 'gold', 'tree', 'true', 'road', 'rock']

In [4]:
# 배치 데이터 생성
def make_batch(word_list):
    
    input_batch = []
    target_batch = []

    for word in word_list:
        # 입력 단어를 인덱스로 변환
        input = [char_to_idx[c] for c in word[:-1]]
        
        # 목표 캐릭터를 인덱스로 변환
        target = char_to_idx[word[-1]]

        # 입력 인덱스를 원핫인코딩으로 변환
        input_batch.append(np.eye(dic_len)[input])

        # 목표 인덱스를 원핫인코딩으로 변환
        target_batch.append(np.eye(dic_len)[target])

    return np.array(input_batch), np.array(target_batch)

In [5]:
# 입력/목표 배치 생성
x_train, y_train = make_batch(word_list)

In [6]:
x_train[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [7]:
y_train[0]


array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

# 모델 생성

In [8]:
# 세 글자만큼 타임스텝 반복
time_step = 3

def build_model():
    
    model = models.Sequential()
    model.add(layers.LSTM(64, input_shape = (time_step, dic_len)))
    model.add(layers.Dense(dic_len, activation='softmax'))

    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['acc'])
    
    return model   

# 훈련 및 테스트

In [10]:
# 모델 생성
model = build_model()

# 훈련 시작
model.fit(x_train,
          y_train,
          epochs=100,
          batch_size=1)


Epoch 1/100
12/12 [==============================] - 1s 45ms/step - loss: 3.2309 - acc: 0.0833
Epoch 2/100
12/12 [==============================] - 0s 5ms/step - loss: 3.1384 - acc: 0.5000
Epoch 3/100
12/12 [==============================] - 0s 5ms/step - loss: 3.0405 - acc: 0.5833
Epoch 4/100
12/12 [==============================] - 0s 5ms/step - loss: 2.9101 - acc: 0.6667
Epoch 5/100
12/12 [==============================] - 0s 5ms/step - loss: 2.7252 - acc: 0.5000
Epoch 6/100
12/12 [==============================] - 0s 5ms/step - loss: 2.4670 - acc: 0.5000
Epoch 7/100
12/12 [==============================] - 0s 5ms/step - loss: 2.1316 - acc: 0.5000
Epoch 8/100
12/12 [==============================] - 0s 5ms/step - loss: 1.7681 - acc: 0.5000
Epoch 9/100
12/12 [==============================] - 0s 5ms/step - loss: 1.5018 - acc: 0.5000
Epoch 10/100
12/12 [==============================] - 0s 5ms/step - loss: 1.3474 - acc: 0.5000
Epoch 11/100
12/12 [==============================] - 0s 5

In [11]:
# 훈련셋 데이터 예측
# 26개 캐릭터의 원핫인코딩 형식
results = model.predict(x_train)
results


array([[5.50502222e-09, 4.74356243e-09, 4.04384082e-09, 4.96363675e-04,
        9.49670970e-01, 5.97114491e-09, 3.87422716e-09, 8.41021475e-09,
        4.13837942e-09, 5.08541254e-09, 4.87111919e-02, 3.25193361e-09,
        5.18551557e-09, 6.54680932e-09, 2.25573027e-09, 4.60557015e-09,
        7.62991537e-09, 2.73120948e-09, 5.29540101e-09, 1.12129853e-03,
        6.39206732e-09, 6.18729112e-09, 2.92639002e-09, 4.72743977e-09,
        4.23271462e-09, 7.18073778e-09],
       [9.55346557e-09, 9.29253297e-09, 6.99690750e-09, 8.02447461e-03,
        1.07272476e-01, 9.12251963e-09, 7.54979190e-09, 1.63752336e-08,
        8.04629785e-09, 8.92133301e-09, 8.80242705e-01, 7.54676854e-09,
        8.48177528e-09, 1.09040963e-08, 4.78201745e-09, 9.45930623e-09,
        1.47046402e-08, 4.82903761e-09, 8.92318752e-09, 4.46020113e-03,
        1.09384244e-08, 1.46884940e-08, 7.39404138e-09, 8.12654566e-09,
        8.05030442e-09, 1.75797243e-08],
       [4.73548845e-08, 3.78059823e-08, 2.49046277e-08

In [12]:
# 1축을 기준으로 최대값의 인덱스 구함
results = np.argmax(results, 1) 
results


array([ 4, 10,  4, 19,  4,  4,  3,  3,  4,  4,  3, 10], dtype=int64)

In [13]:
# 예측 결과 출력
for i, word in enumerate(word_list):
    last_char = char_list[results[i]]
    print(word[:3] + ' -> ' + word[:3] + last_char)

lov -> love
loo -> look
fac -> face
fas -> fast
hom -> home
hop -> hope
goo -> good
gol -> gold
tre -> tree
tru -> true
roa -> road
roc -> rock
